In [1]:
import yfinance as yf
import datetime

In [ ]:
def get_option_chain(ticker: str, expiry: str):

    """
    Fetches the option chain (calls and puts) for a given stock ticker and expiry date.

    Parameters
    ----------
    ticker : str
        Stock ticker symbol (e.g., 'AAPL')
    expiry : str
        Expiration date in 'YYYY-MM-DD' format

    Returns
    -------
    tuple[pd.DataFrame, pd.DataFrame]
        Calls DataFrame, Puts DataFrame
    """
    
    try:
        stock = yf.Ticker(ticker)

        if expiry not in stock.options:
            raise ValueError(f"Expiration `{expiry}` cannot be found. Available expirations are: {stock.options}")

        option_chain = stock.option_chain(expiry)
        calls = option_chain.calls.copy()
        puts = option_chain.puts.copy()

        # Add option type labels for downstream processing
        calls['option_type'] = 'call'
        puts['option_type'] = 'put'

        # Add Time To Maturity (TTM) in years
        expiry_date = datetime.strptime(expiry, "%Y-%m-%d").date()
        today = datetime.today().date()
        ttm = max((expiry_date - today).days/365.0, 0)

        calls['TTM'] = ttm
        puts['TTM'] = ttm


        return calls, puts
    except Exception as e:
        print(f"Error fetching option chain for {ticker} on {expiry}: {e}")
        return pd.DataFrame(), pd.DataFrame() # Return two empty frames so pipeline doesn't break
